# Instacart Feature Engineering

In [1]:
#import data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#data types
import collections

#import general libraries
import gc
import time
import random
import datetime

#deep learning 
import tensorflow as tf

/Users/waficel-assi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', -1)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Read and merge data

In [3]:
#read data
order_products_prior_df = pd.read_csv('./data/order_products__prior.csv', engine='c',
                                        dtype={'order_id': np.int32, 'product_id': np.int32,
                                        'add_to_cart_order': np.int16, 'reordered': np.int8})
print('shape of order_products_prior_df is {}'.format(order_products_prior_df.shape))

order_products_train_df = pd.read_csv('./data/order_products__train.csv', engine='c',
                                      dtype={'order_id': np.int32, 'product_id': np.int32,
                                             'add_to_cart_order': np.int16, 'reordered': np.int8})
print('shape of order_products_train_df is {}'.format(order_products_train_df.shape))

orders_df = pd.read_csv('./data/orders.csv', engine='c',
                        dtype={'order_id': np.int32, 'user_id': np.int32, 'order_number': np.int32,
                               'order_dow': np.int8, 'order_hour_of_day': np.int8,
                               'days_since_prior_order': np.float16})
print('shape of orders_df is {}'.format(orders_df.shape))

products_df = pd.read_csv("./data/products.csv", engine='c')
print('shape of products_df is {}'.format(products_df.shape))

aisles = pd.read_csv("./data/aisles.csv")
print('shape of aisles is {}'.format(aisles.shape))

departments = pd.read_csv("./data/departments.csv")
print('shape of departments is {}'.format(departments.shape))

#merge data
df_products_orders_all = pd.concat([order_products_prior_df, order_products_train_df])

print('merging order_products with orders')

df_products_orders_all = pd.merge(df_products_orders_all, orders_df, on='order_id')
print('shape of df_products_orders_all is {}'.format(df_products_orders_all.shape))

print('grabbing aisles and departments names ')

df = df_products_orders_all.merge(products_df, how='inner', on='product_id')
df = df.merge(aisles, how='inner', on='aisle_id')
df = df.merge(departments, how='inner', on='department_id')
print('shape of df is {}'.format(df.shape))

print('sorting by user and order...')

df = df.drop(columns=['aisle_id','department_id','product_name'], axis=1)
df.sort_values(['user_id', 'order_number', 'add_to_cart_order'], axis=0, inplace=True)
df = df.reset_index(drop=True)
df.head()

shape of order_products_prior_df is (32434489, 4)
shape of order_products_train_df is (1384617, 4)
shape of orders_df is (3421083, 7)
shape of products_df is (49688, 4)
shape of aisles is (134, 2)
shape of departments is (21, 2)
merging order_products with orders
shape of df_products_orders_all is (33819106, 10)
grabbing aisles and departments names 
shape of df is (33819106, 15)
sorting by user and order...


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department
0,2539329,196,1,0,1,prior,1,2,8,NaN,soft drinks,beverages
1,2539329,14084,2,0,1,prior,1,2,8,NaN,soy lactosefree,dairy eggs
2,2539329,12427,3,0,1,prior,1,2,8,NaN,popcorn jerky,snacks
3,2539329,26088,4,0,1,prior,1,2,8,NaN,popcorn jerky,snacks
4,2539329,26405,5,0,1,prior,1,2,8,NaN,paper goods,household


In [4]:
del order_products_prior_df
del order_products_train_df
del products_df
del aisles
del departments
gc.collect()

28

## Feature engineering

In [5]:
user_order_basket_size=pd.DataFrame(df.groupby(['user_id','order_id']).size()).rename(columns={0:'basket_size'}).reset_index()
basket_size = user_order_basket_size.merge(df, how='left', on=['user_id','order_id'])['basket_size']

In [6]:
# count of products, aisles and departments per order
basket_stats = (df
                .groupby(['user_id', 'order_id'])
                .agg({'product_id':'count','department':'nunique', 'aisle':'nunique'})
                .reset_index()
                .rename(columns={'product_id':'count_products',
                                 'department':'count_departments'
                                 ,'aisle':'count_aisles'}))

df = df.merge(basket_stats, how='left', on=['user_id','order_id'])

gc.collect()

In [7]:
# count of orders per user
orders_per_user = df.groupby('user_id').agg({'order_id':'nunique'}).reset_index().rename(columns={'order_id':'total_orders_per_user'})
df = df.merge(orders_per_user, how='left', on=['user_id'])
df.head()

gc.collect()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department,count_products,count_departments,count_aisles,total_orders_per_user
0,2539329,196,1,0,1,prior,1,2,8,NaN,soft drinks,beverages,5,4,4,11
1,2539329,14084,2,0,1,prior,1,2,8,NaN,soy lactosefree,dairy eggs,5,4,4,11
2,2539329,12427,3,0,1,prior,1,2,8,NaN,popcorn jerky,snacks,5,4,4,11
3,2539329,26088,4,0,1,prior,1,2,8,NaN,popcorn jerky,snacks,5,4,4,11
4,2539329,26405,5,0,1,prior,1,2,8,NaN,paper goods,household,5,4,4,11


In [8]:
# cumsum of user lifetime
df.days_since_prior_order = df.days_since_prior_order.fillna(0)
cum_days_since_prior_order = df.groupby(['user_id','order_id']).agg({'days_since_prior_order':'mean'}).reset_index()
cum_days_since_prior_order = cum_days_since_prior_order.sort_values(by=['user_id','days_since_prior_order'])
cum_days_since_prior_order['cum_days_since_prior_order'] = cum_days_since_prior_order.groupby(['user_id'])['days_since_prior_order'].cumsum()

df = df.merge(cum_days_since_prior_order[['user_id','order_id','cum_days_since_prior_order']], how='left', on=['user_id','order_id'])


# user lifetime (how long have they been ordering)
customer_lifetime = (cum_days_since_prior_order
                        .groupby('user_id')
                        .agg({'cum_days_since_prior_order': 'max'})
                        .rename(columns={'cum_days_since_prior_order':'max_cum_days_since_prior_order'})
                        .reset_index())

df = df.merge(customer_lifetime, how='left', on=['user_id'])
df.head()
gc.collect()

70

In [13]:
products_dow_stats = df.groupby(['user_id','order_dow']).agg({'order_dow':'count'}).rename(columns={'order_dow':'count_products_ordered_dow'}).reset_index()
#products_hod_stats = df.groupby(['user_id','order_hour_of_day']).agg({'order_hour_of_day':'count'}).rename(columns={'order_hour_of_day':'count_products_ordered_hod'}).reset_index()

df = df.merge(products_dow_stats, how='left', on=['user_id','order_dow'])
#df = df.merge(df, how='left', on=['user_id','order_hour_of_day'])

gc.collect()

63

In [15]:
# user_id order hour of day stats
orders_hod_stats = df.groupby(['user_id', 'order_hour_of_day']).agg({'order_hour_of_day':['count','mean']})

# user_id order day of week stats
orders_dow_stats = df.groupby(['user_id', 'order_dow']).agg({'order_dow':['count','mean']})

# user_id basket_stats
user_id_basket_stats = (basket_stats
                            .groupby('user_id')
                            .agg({'count_products':['sum','mean','max','min'],
                                'count_departments':['mean','max','min'], 
                                'count_aisles':['mean','max','min']}))

#Number of items reordered per user
products_per_user = (df.groupby('user_id').agg({'reordered':['sum', 'mean']}))

#feature engineering from orders dataset
days_between_orders = (orders_df
                           .groupby('user_id')
                           .agg({'days_since_prior_order': ['min','max','mean']}))

# add_to_cart_order stats (mean, min, max)
product_cart_order = df.groupby(['user_id','product_id']).agg({'add_to_cart_order': ['min','max','mean']})

gc.collect()

56

In [33]:
list_groupby_features = [orders_hod_stats, orders_dow_stats, 
                         user_id_basket_stats, products_per_user, 
                         days_between_orders, product_cart_order]

for groupby_df in list_groupby_features:
    groupby_df.columns = pd.Index([e[0] + "_" + e[1].upper() for e in groupby_df.columns.tolist()])

gc.collect()

3104

In [38]:
df = df.merge(orders_hod_stats, on=['user_id', 'order_hour_of_day'])
df = df.merge(orders_dow_stats, on=['user_id', 'order_dow'])
df = df.merge(user_id_basket_stats, on='user_id')
df = df.merge(products_per_user, on='user_id')
df = df.merge(days_between_orders, on='user_id')

gc.collect()

215

In [39]:
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department,count_products,count_departments,count_aisles,total_orders_per_user,cum_days_since_prior_order,max_cum_days_since_prior_order,count_products_ordered_dow_x,count_products_ordered_dow_y,order_hour_of_day_COUNT,order_hour_of_day_MEAN,order_dow_COUNT,order_dow_MEAN,count_products_SUM,count_products_MEAN,count_products_MAX,count_products_MIN,count_departments_SUM,count_departments_MEAN,count_departments_MAX,count_departments_MIN,count_aisles_SUM,count_aisles_MEAN,count_aisles_MAX,count_aisles_MIN,reordered_SUM,reordered_MEAN,days_since_prior_order_MIN,days_since_prior_order_MAX,days_since_prior_order_MEAN
0,2539329,196,1,0,1,prior,1,2,8,0.0,soft drinks,beverages,5,4,4,11,0.0,190.0,9,9,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0
1,2539329,14084,2,0,1,prior,1,2,8,0.0,soy lactosefree,dairy eggs,5,4,4,11,0.0,190.0,9,9,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0
2,2539329,12427,3,0,1,prior,1,2,8,0.0,popcorn jerky,snacks,5,4,4,11,0.0,190.0,9,9,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0
3,2539329,26088,4,0,1,prior,1,2,8,0.0,popcorn jerky,snacks,5,4,4,11,0.0,190.0,9,9,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0
4,2539329,26405,5,0,1,prior,1,2,8,0.0,paper goods,household,5,4,4,11,0.0,190.0,9,9,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0


In [43]:
user_product_reorders = df.groupby(['user_id','product_id']).agg({'reordered':'sum'}).rename(columns={'reordered':'sum_product_reorders_by_user'}).reset_index()
product_reorder = df.groupby('product_id').agg({'reordered':'sum'}).rename(columns={'reordered':'sum_total_product_reorders'}).reset_index()

In [44]:
df = df.merge(user_product_reorders, on=['user_id','product_id'])
df = df.merge(product_reorder, on=['product_id'])

In [46]:
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department,count_products,count_departments,count_aisles,total_orders_per_user,cum_days_since_prior_order,max_cum_days_since_prior_order,count_products_ordered_dow_x,count_products_ordered_dow_y,order_hour_of_day_COUNT,order_hour_of_day_MEAN,order_dow_COUNT,order_dow_MEAN,count_products_SUM,count_products_MEAN,count_products_MAX,count_products_MIN,count_departments_SUM,count_departments_MEAN,count_departments_MAX,count_departments_MIN,count_aisles_SUM,count_aisles_MEAN,count_aisles_MAX,count_aisles_MIN,reordered_SUM,reordered_MEAN,days_since_prior_order_MIN,days_since_prior_order_MAX,days_since_prior_order_MEAN,sum_product_reorders_by_user,sum_total_product_reorders
0,2539329,196,1,0,1,prior,1,2,8,0.0,soft drinks,beverages,5,4,4,11,0.0,190.0,9,9,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0,10,29012.0
1,2398795,196,1,1,1,prior,2,3,7,15.0,soft drinks,beverages,6,4,5,11,43.0,190.0,11,11,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0,10,29012.0
2,473747,196,1,1,1,prior,3,3,12,21.0,soft drinks,beverages,5,4,5,11,103.0,190.0,11,11,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0,10,29012.0
3,2254736,196,1,1,1,prior,4,4,7,29.0,soft drinks,beverages,5,4,5,11,160.0,190.0,33,33,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0,10,29012.0
4,431534,196,1,1,1,prior,5,4,15,28.0,soft drinks,beverages,8,4,5,11,131.0,190.0,33,33,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0,10,29012.0


In [47]:
df[df.user_id==1].shape

(70, 43)

In [53]:
#reordered_MEAN == user_reorder_rate
df = (df
      .rename(columns = {'reordered_SUM':'sum_reordered_products_by_user',
                        'reordered_MEAN':'user_reorder_probability'})
      .assign(
        user_product_reorder_probability = lambda df: df.sum_product_reorders_by_user/df.sum_reordered_products_by_user,
        product_reorder_probability = lambda df: df.sum_total_product_reorders/df.shape[0]
    ))

gc.collect()
      
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department,count_products,count_departments,count_aisles,total_orders_per_user,cum_days_since_prior_order,max_cum_days_since_prior_order,count_products_ordered_dow_x,count_products_ordered_dow_y,order_hour_of_day_COUNT,order_hour_of_day_MEAN,order_dow_COUNT,order_dow_MEAN,count_products_SUM,count_products_MEAN,count_products_MAX,count_products_MIN,count_departments_SUM,count_departments_MEAN,count_departments_MAX,count_departments_MIN,count_aisles_SUM,count_aisles_MEAN,count_aisles_MAX,count_aisles_MIN,sum_reordered_products_by_user,user_reorder_probability,days_since_prior_order_MIN,days_since_prior_order_MAX,days_since_prior_order_MEAN,sum_product_reorders_by_user,sum_total_product_reorders,user_product_reorder_probability,product_reorder_probability
0,2539329,196,1,0,1,prior,1,2,8,0.0,soft drinks,beverages,5,4,4,11,0.0,190.0,9,9,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0,10,29012.0,0.196078,0.000858
1,2398795,196,1,1,1,prior,2,3,7,15.0,soft drinks,beverages,6,4,5,11,43.0,190.0,11,11,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0,10,29012.0,0.196078,0.000858
2,473747,196,1,1,1,prior,3,3,12,21.0,soft drinks,beverages,5,4,5,11,103.0,190.0,11,11,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0,10,29012.0,0.196078,0.000858
3,2254736,196,1,1,1,prior,4,4,7,29.0,soft drinks,beverages,5,4,5,11,160.0,190.0,33,33,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0,10,29012.0,0.196078,0.000858
4,431534,196,1,1,1,prior,5,4,15,28.0,soft drinks,beverages,8,4,5,11,131.0,190.0,33,33,70,10.142857,70,2.857143,70,6.363636,11,4,42,3.818182,5,3,61,5.545455,10,4,51.0,0.728571,0.0,30.0,19.0,10,29012.0,0.196078,0.000858
